In [52]:
import pandas as pd
import re
import nltk
import string
import google_conf
from importlib import reload

In [53]:
pop_data = google_conf.setup(sheet_url="https://docs.google.com/spreadsheets/d/1o8L0Rt4CzQ_DcYdaKgCdige4nJs0sydlTGiMBSstQtc/edit?usp=sharing", service_account_path="../../ServiceAccountsKey.json")

In [54]:
articles_metadata = pd.read_json("../data/article_metadata_with_filepaths.json")
articles_metadata.head(5)

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath,pop_id
0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,y,y,y,None,[Political Communication/2011 - Bos et al. - H...,0
1,2019,Blassnig et al.,Political Communication,Hitting a Nerve: Populist News Articles Lead t...,article,y,y,y,None,[Political Communication/2019 - Blassing et al...,1
2,2017,Caramani,American Political Science Review,Will vs. Reason: The Populist and Technocratic...,article,y,y,None,None,[American Political Science Review/2017 - Cara...,2
3,2020,"Wuttke,Schimpf, Schoen",American Political Science Review,When the Whole Is Greater than the Sum of Its ...,article,y,y,None,None,[American Political Science Review/2020 - Wutt...,3
4,2014,Treib,Journal of European Public Policy,"The voter says no, but nobody listens: causes ...",article,y,y,None,None,[Journal of European Public Policy/2014 - Trei...,4


In [55]:
articles_metadata = articles_metadata[articles_metadata["filepath"].apply(bool)].copy()

In [56]:
sample_pop_ids = [62, 377, 188, 364, 499, 228, 376, 90, 227, 330]

# Test with one document

In [57]:
pop_id = 90
path = "../data/large_data/articles_filteredtexts/"
filename = "filteredtext_pop_id_{}.txt".format(str(pop_id))

In [58]:
with open(path + filename, "r") as f:
    filteredtext = f.read()

In [59]:
test_str_list = filteredtext.split()[:100]
print(test_str_list)

['Abstract.', 'As', 'a', 'result', 'of', 'the', 'steady', 'rise', 'of', 'populist', 'parties', 'and', 'politicians', 'all', 'over', 'the', 'world', '–', 'and', 'particularly', 'since', 'the', 'Brexit', 'referendum', 'and', 'the', 'election', 'of', 'Donald', 'Trump', '–', 'populism', 'research', 'has', 'become', 'increasingly', 'popular', 'and', 'widespread.', 'The', 'field,', 'however,', 'also', 'faces', 'some', 'tricky', 'challenges.', 'First,', 'it', 'is', 'easy', 'to', 'confuse', 'populism', 'with', 'related', 'concepts', 'like,', 'for', 'instance,', '‘nativism’', 'and', '‘Euroscepticism’.', 'This', 'brings', 'the', 'risk', 'of', 'sloppy', 'conceptualisation,', 'and,', 'as', 'a', 'result,', 'invalid', 'inferences.', 'Second,', 'populism', 'research', 'remains', 'relatively', 'detached', 'from', 'adjacent', 'fields,and', 'fruitful', 'fertilisation', 'across', 'literatures', 'is', 'still', 'rather', 'uncommon.In', 'order', 'to', 'deal', 'with', 'these', 'challenges,populism', 'researc

In [60]:
# gather indeces:
filteredtext_split = filteredtext.split()
inxs = [t[0] for t in enumerate(filteredtext_split) if re.search("populis", t[1], re.IGNORECASE)]
inxs

[9,
 31,
 53,
 77,
 98,
 129,
 171,
 181,
 183,
 222,
 224,
 253,
 294,
 306,
 331,
 333,
 378,
 387,
 408,
 474,
 498,
 506,
 515,
 524,
 551,
 597,
 608,
 625,
 649,
 664,
 678,
 741,
 770,
 838,
 858,
 865,
 882,
 945,
 968,
 1012,
 1026,
 1048,
 1057,
 1100,
 1143,
 1157,
 1162,
 1174,
 1196,
 1241,
 1280,
 1289,
 1326,
 1339,
 1346,
 1379,
 1402,
 1415,
 1418,
 1439,
 1457,
 1526,
 1569,
 1588,
 1603,
 1618,
 1636,
 1657,
 1698,
 1755,
 1770,
 1818,
 1841,
 1851,
 1875,
 1889,
 1925,
 1939,
 1959,
 2025,
 2036,
 2088,
 2117,
 2139,
 2157,
 2177,
 2186,
 2192,
 2249,
 2259,
 2288,
 2367,
 2460,
 2495,
 2525,
 2561,
 2653,
 2663,
 2684,
 2695,
 2714,
 2726,
 2762,
 2827,
 2923,
 2947,
 2979,
 3001,
 3020,
 3113,
 3120,
 3172,
 3237,
 3269,
 3280,
 3299,
 3343,
 3352,
 3528,
 3544,
 3592,
 3651,
 3711,
 3753,
 3776,
 3787,
 3811,
 3854,
 3892,
 3916,
 3957,
 3978,
 4006,
 4019,
 4047,
 4085]

In [61]:
for i in inxs:
    print(filteredtext_split[i-2:i+3])


['rise', 'of', 'populist', 'parties', 'and']
['Trump', '–', 'populism', 'research', 'has']
['to', 'confuse', 'populism', 'with', 'related']
['inferences.', 'Second,', 'populism', 'research', 'remains']
['with', 'these', 'challenges,populism', 'research', 'should']
['other', 'researchers,', 'populism', 'scholars', 'should']
['can', 'inspire', 'populism', 'scholars', 'to']
['expectations.', 'Keyword:', 'populism', 'Introduction', 'Populism']
['populism', 'Introduction', 'Populism', 'is', 'sexy.Particularly']
['the', 'terms', '‘populism’', 'or', '‘populist’']
['‘populism’', 'or', '‘populist’', '671', 'times.']
['in', '2017', '‘populism’', 'was', 'declared']
['rise', 'of', 'populist', 'parties', 'and']
['appeal', 'of', 'populism', 'is', 'also']
['the', 'words', '‘populism’', 'or', '‘populist’']
['‘populism’', 'or', '‘populist’', 'in', 'the']
['devoted', 'to', 'populism!', 'It', 'seems']
['research', 'on', 'populism', 'has', 'become']
['literature', 'on', 'populism,', 'thereby', 'mainly']
[

In [62]:
def extract_indeces(pop_id):
    path = "../data/large_data/articles_filteredtexts/"
    filename = "filteredtext_pop_id_{}.txt".format(str(pop_id))
    with open(path + filename, "r") as f:
        filteredtext = f.read()
    filteredtext_split = filteredtext.split()
    inxs = [t[0] for t in enumerate(filteredtext_split) if re.search("populis", t[1], re.IGNORECASE)]
    return inxs
articles_metadata["populism_inxs"] = articles_metadata["pop_id"].apply(extract_indeces)

In [63]:
articles_metadata["populism_inxs"]

0      [23, 37, 75, 178, 199, 225, 229, 231, 236, 302...
1      [17, 26, 40, 49, 104, 128, 144, 145, 161, 223,...
2      [4, 19, 59, 100, 125, 339, 466, 541, 633, 673,...
3      [65, 125, 142, 195, 225, 254, 282, 304, 313, 3...
4              [249, 2269, 3843, 3951, 4395, 4437, 4558]
                             ...                        
537                               [133, 216, 2402, 3622]
538    [483, 493, 501, 523, 687, 1104, 1138, 1151, 11...
539    [20, 66, 285, 355, 361, 421, 609, 620, 680, 72...
540    [41, 177, 199, 240, 828, 855, 974, 1515, 2746,...
541    [2, 8, 12, 66, 81, 95, 100, 117, 139, 148, 178...
Name: populism_inxs, Length: 537, dtype: object

In [64]:
articles_metadata["populis*_N"] = articles_metadata["populism_inxs"].apply(len)

In [65]:
articles_metadata["populis*_N"].sum()

41084

In [127]:
replace_tups = [
    ("populists", "populis*"),
    ("populisms", "populis*"),
    ("populist", "populis*"),
    ("populism", "populis*"),
]
def clean_conc(conc_string):
    conc_cleaned = " ".join([re.sub("\W", "", w, count=10) for w in conc_string.lower().split()])
    conc_cleaned = re.sub("\d+", "", conc_cleaned, count=10)
    #conc_cleaned = conc_cleaned.lower().translate(str.maketrans("", "", string.punctuation))
    for tup in replace_tups:
        conc_cleaned = conc_cleaned.replace(tup[0], tup[1])
    return conc_cleaned

In [128]:
clean_conc("4right-wing populists.12@#`^&")

'rightwing populis*'

In [129]:
def get_concordance(row, left=1, right=1):
    pop_id = row["pop_id"]
    path = "../data/large_data/articles_filteredtexts/"
    filename = "filteredtext_pop_id_{}.txt".format(str(pop_id))
    with open(path + filename, "r") as f:
        filteredtext = f.read()
    filteredtext_split = filteredtext.split()
    inxs = row["populism_inxs"]
    conc_data = [clean_conc(" ".join(filteredtext_split[i-left: i+right+1])) for i in inxs]
    return conc_data

In [130]:
articles_metadata["conc_left1"] = articles_metadata.apply(lambda row: get_concordance(row, left=1, right=0), axis=1)
articles_metadata["conc_right1"] = articles_metadata.apply(lambda row: get_concordance(row, left=0, right=1), axis=1)

In [131]:
articles_metadata

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath,pop_id,populism_inxs,populis*_N,conc_left1,conc_right1,conc_left1_interesting
0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,y,y,y,None,[Political Communication/2011 - Bos et al. - H...,0,"[23, 37, 75, 178, 199, 225, 229, 231, 236, 302...",199,"[rightwing populis*, rightwing populis*, right...","[populis* leaders, populis* party, populis* le...","{'rightwing populis*': 109, 'leftwing populis*..."
1,2019,Blassnig et al.,Political Communication,Hitting a Nerve: Populist News Articles Lead t...,article,y,y,y,None,[Political Communication/2019 - Blassing et al...,1,"[17, 26, 40, 49, 104, 128, 144, 145, 161, 223,...",222,"[nerve populis*, more populis*, of populis*, h...","[populis* news, populis* reader, populis* comm...","{'rightwing populis*': 3, 'leftwing populis*':..."
2,2017,Caramani,American Political Science Review,Will vs. Reason: The Populist and Technocratic...,article,y,y,None,None,[American Political Science Review/2017 - Cara...,2,"[4, 19, 59, 100, 125, 339, 466, 541, 633, 673,...",117,"[analytically populis*, that populis*, both po...","[populis* and, populis* and, populis* and, pop...","{'rightwing populis*': 2, 'leftwing populis*':..."
3,2020,"Wuttke,Schimpf, Schoen",American Political Science Review,When the Whole Is Greater than the Sum of Its ...,article,y,y,None,None,[American Political Science Review/2020 - Wutt...,3,"[65, 125, 142, 195, 225, 254, 282, 304, 313, 3...",296,"[of populis*, both conceptuallyandempiricallyt...","[populis* attitudes, conceptuallyandempiricall...","{'rightwing populis*': 0, 'leftwing populis*':..."
4,2014,Treib,Journal of European Public Policy,"The voter says no, but nobody listens: causes ...",article,y,y,None,None,[Journal of European Public Policy/2014 - Trei...,4,"[249, 2269, 3843, 3951, 4395, 4437, 4558]",7,"[euroscepticism populis*, on populis*, on popu...","[populis* secondorder, populis* and, populis* ...","{'rightwing populis*': 0, 'leftwing populis*':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,2018,Grzymala-Busse,Current History,Poland’s Path to Illiberalism,article,y,y,None,None,[Current History/2018 - Grzymala-Busse - Polan...,537,"[133, 216, 2402, 3622]",4,"[illiberal populis*, of populis*, often populi...","[populis* it, populis* pis, populis* and, popu...","{'rightwing populis*': 0, 'leftwing populis*':..."
538,2017,Eichengreen,Current History,The Populist Turn in American Politics,article,y,y,None,None,[Current History/2017 - Eichengreen - The Popu...,538,"[483, 493, 501, 523, 687, 1104, 1138, 1151, 11...",20,"[new populis*, old populis*, the populis*, po...","[populis* bears, populis* of, populis* revolt,...","{'rightwing populis*': 0, 'leftwing populis*':..."
539,2014,Mudde,Current History,The Far Right and the European Elections,article,y,y,None,None,[Current History/2014 - Mudde - The Far Right ...,539,"[20, 66, 285, 355, 361, 421, 609, 620, 680, 72...",17,"[antieuropean populis*, european populis*, ant...","[populis* in, populis* backlash, populis* are,...","{'rightwing populis*': 0, 'leftwing populis*':..."
540,2020,Cano,European History Quarterly,Printing the People,article,y,y,None,None,[European History Quarterly/2020 - Cano - Prin...,540,"[41, 177, 199, 240, 828, 855, 974, 1515, 2746,...",39,"[this populis*, nationalism populis*, photogra...","[populis* nationalism, populis* and, populis* ...","{'rightwing populis*': 0, 'leftwing populis*':..."


In [132]:
conc_left1_all = []
for conc_list in articles_metadata["conc_left1"]:
    conc_left1_all.extend(conc_list)

In [133]:
len(conc_left1_all)

41084

In [134]:
conc_left1_all_df = pd.DataFrame(nltk.FreqDist(conc_left1_all).most_common(), columns=["conc", "count"])
conc_left1_all_df[:100]

,conc,count
0,of populis*,7486
1,the populis*,3217
2,rightwing populis*,1680
3,that populis*,1415
4,a populis*,1382
...,...,...
95,however populis*,37
96,illiberal populis*,37
97,such populis*,36
98,but populis*,36


In [137]:
google_conf.set_with_dataframe(pop_data.add_worksheet("conc_left1_v3", 1,1), conc_left1_all_df)

In [138]:
conc_left1_interesting = ['rightwing populis*', 'leftwing populis*', 'right populis*', 'american populis*', 'left populis*', 'new populis*', 'european populis*', 'contemporary populis*', 'radical populis*', 'authoritarian populis*', 'class populis*', 'strong populis*', 'exclusionary populis*', 'economic populis*', 'leftist populis*', 'political populis*', 'radicalright populis*', 'japanese populis*']
conc_left1_interesting

['rightwing populis*',
 'leftwing populis*',
 'right populis*',
 'american populis*',
 'left populis*',
 'new populis*',
 'european populis*',
 'contemporary populis*',
 'radical populis*',
 'authoritarian populis*',
 'class populis*',
 'strong populis*',
 'exclusionary populis*',
 'economic populis*',
 'leftist populis*',
 'political populis*',
 'radicalright populis*',
 'japanese populis*']

In [139]:
conc_left1_interesting_df = conc_left1_all_df[conc_left1_all_df["conc"].isin(conc_left1_interesting)]
conc_left1_interesting_df

,conc,count
2,rightwing populis*,1680
14,leftwing populis*,388
16,right populis*,223
17,american populis*,221
21,new populis*,165
22,left populis*,160
25,european populis*,130
26,contemporary populis*,126
35,authoritarian populis*,96
40,radical populis*,85


In [141]:
google_conf.set_with_dataframe(pop_data.add_worksheet("conc_left1_interesting_v2",1,1), conc_left1_interesting_df)

In [142]:
def ectract_counts_of_interesting(list_of_concs):
    conc_count_dict = dict([(conc, list_of_concs.count(conc)) for conc in conc_left1_interesting])
    return conc_count_dict
articles_metadata["conc_left1_interesting"] = articles_metadata["conc_left1"].apply(ectract_counts_of_interesting)

In [143]:
articles_metadata.head(20)

,year,authors,journal,title,type,pdf?,keywords,keywords_online?,notes,filepath,pop_id,populism_inxs,populis*_N,conc_left1,conc_right1,conc_left1_interesting
0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,y,y,y,None,[Political Communication/2011 - Bos et al. - H...,0,"[23, 37, 75, 178, 199, 225, 229, 231, 236, 302...",199,"[rightwing populis*, rightwing populis*, right...","[populis* leaders, populis* party, populis* le...","{'rightwing populis*': 111, 'leftwing populis*..."
1,2019,Blassnig et al.,Political Communication,Hitting a Nerve: Populist News Articles Lead t...,article,y,y,y,None,[Political Communication/2019 - Blassing et al...,1,"[17, 26, 40, 49, 104, 128, 144, 145, 161, 223,...",222,"[nerve populis*, more populis*, of populis*, h...","[populis* news, populis* reader, populis* comm...","{'rightwing populis*': 3, 'leftwing populis*':..."
2,2017,Caramani,American Political Science Review,Will vs. Reason: The Populist and Technocratic...,article,y,y,None,None,[American Political Science Review/2017 - Cara...,2,"[4, 19, 59, 100, 125, 339, 466, 541, 633, 673,...",117,"[analytically populis*, that populis*, both po...","[populis* and, populis* and, populis* and, pop...","{'rightwing populis*': 2, 'leftwing populis*':..."
3,2020,"Wuttke,Schimpf, Schoen",American Political Science Review,When the Whole Is Greater than the Sum of Its ...,article,y,y,None,None,[American Political Science Review/2020 - Wutt...,3,"[65, 125, 142, 195, 225, 254, 282, 304, 313, 3...",296,"[of populis*, both conceptuallyandempiricallyt...","[populis* attitudes, conceptuallyandempiricall...","{'rightwing populis*': 0, 'leftwing populis*':..."
4,2014,Treib,Journal of European Public Policy,"The voter says no, but nobody listens: causes ...",article,y,y,None,None,[Journal of European Public Policy/2014 - Trei...,4,"[249, 2269, 3843, 3951, 4395, 4437, 4558]",7,"[euroscepticism populis*, on populis*, on popu...","[populis* secondorder, populis* and, populis* ...","{'rightwing populis*': 0, 'leftwing populis*':..."
5,2016,Hobolt,Journal of European Public Policy,"The Brexit vote: a divided nation, a divided c...",article,y,y,None,None,[Journal of European Public Policy/2016 - Hobo...,5,"[153, 515, 717, 923, 3225, 3259, 3816, 4125, 4...",17,"[euroscepticism populis*, of populis*, the pop...","[populis* referendum, populis* parties, populi...","{'rightwing populis*': 0, 'leftwing populis*':..."
6,2018,Kreuder-Sonnen,Journal of European Public Policy,An authoritarian turn in Europe and European S...,article,y,y,None,None,[Journal of European Public Policy/2018 - Kreu...,6,"[121, 183, 518, 529, 648, 678, 1712, 2347, 249...",25,"[nationalist populis*, legitimacy populis*, na...","[populis* have, populis* introduction, populis...","{'rightwing populis*': 2, 'leftwing populis*':..."
7,2019,"McDonnell, Werner",Journal of European Public Policy,Differently Eurosceptic: radical right populis...,article,y,y,None,None,[Journal of European Public Policy/2019 - McDo...,7,"[14, 35, 186, 199, 249, 286, 396, 517, 563, 76...",23,"[right populis*, right populis*, right populis...","[populis* parties, populis* rrp, populis* intr...","{'rightwing populis*': 1, 'leftwing populis*':..."
8,2019,Schmidt,Journal of European Public Policy,Politicization in the EU: between national pol...,article,y,y,None,None,[Journal of European Public Policy/2019 - Schm...,8,"[52, 212, 423, 593, 1470, 1968, 2056, 2123, 24...",14,"[euroskeptic populis*, commission populis*, by...","[populis* or, populis* eurozone, populis* chal...","{'rightwing populis*': 0, 'leftwing populis*':..."
9,2020,Bakker et al.,Journal of European Public Policy,"Multidimensional incongruence, political disaf...",article,y,y,None,None,[Journal of European Public Policy/2020 - Bakk...,9,"[197, 1043, 1473, 1595, 1664, 3255, 3287, 3295...",13,"[representation populis*, for populis*, for po...","[populis* research, populis* radical, populis*...","{'rightwing populis*': 0, 'l